In [56]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

from sklearn.gaussian_process import GaussianProcessClassifier, GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from bayes_opt import BayesianOptimization


In [3]:
data = pd.read_csv("./scraped_data/clean_betting_data.csv")
for c in data.columns:
    print(c)

Unnamed: 0
date
season
neutral
playoff
team1
team2
elo1_pre
elo2_pre
elo_prob1
elo_prob2
elo1_post
elo2_post
qbelo1_pre
qbelo2_pre
qb1
qb2
qb1_value_pre
qb2_value_pre
qb1_adj
qb2_adj
qbelo_prob1
qbelo_prob2
qb1_game_value
qb2_game_value
qb1_value_post
qb2_value_post
qbelo1_post
qbelo2_post
score1
score2
quality
importance
total_rating
year
spread
spread_result
ou_line
ou_result


## Preprocess Data

In [29]:
X_data = data[['qbelo1_pre', 'qbelo2_pre', 'qb1_value_pre', 'qb2_value_pre']].values
y_gt = data['ou_result'].values[:,np.newaxis]
enc = OneHotEncoder().fit(y_gt)
y_data = enc.transform(y_gt).toarray()

In [54]:
X_trainval, X_test, y_trainval, y_test = train_test_split(X_data, y_data, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2)

X_mean = np.mean(X_train, axis=0)
X_std = np.std(X_train, axis=0)

X_train = (X_train - X_mean)/X_std
X_val = (X_val - X_mean)/X_std
X_test = (X_test - X_mean)/X_std

## GP Classifier

In [62]:
model = GaussianProcessClassifier(kernel=RBF(0.5))
model = model.fit(X_train, y_train[:,1])
print(model.score(X_val, y_val[:,1]))
print(model.score(X_test, y_test[:,1]))

/Users/clo/miniforge3/envs/nflbetting/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


0.98046875
0.984375
